In [1]:
#==街景字体识别==
#思路：将不定长字符转换为定长字符的识别问题，并使用CNN完成训练和验证
#==task2 数据读取与扩增==

In [2]:
from PIL import Image
import cv
#读图片库Pillow,OpenCV等

In [4]:
#测试读单张图
#OpenCV imread imwrite
#pillow Image.open() im.save()
img = cv2.imread("data//mchar_train//000000.png")
img#读出来的是图片的象素数据


array([[[ 98, 112, 108],
        [ 97, 112, 108],
        [ 98, 114, 107],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[100, 114, 110],
        [ 99, 114, 110],
        [ 99, 115, 108],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[101, 116, 112],
        [101, 116, 112],
        [101, 117, 110],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[ 25,  21,  20],
        [ 24,  22,  21],
        [ 26,  24,  23],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[ 24,  23,  19],
        [ 23,  24,  20],
        [ 22,  23,  19],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[ 21,  22,  18],
        [ 22,  23,  19],
        [ 21,  22,  18],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]]

In [8]:
cv2.imwrite('000.jpg',img)#写出cv2.imwrite

True

In [6]:
edges = cv2.Canny(img,30,70)#cv2.Canny边缘检测
cv2.imwrite('first.jpg',edges)

True

In [9]:
imgGray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)#转换为灰度图2RGB，由BRG转为RGB
cv2.imwrite('000gray.jpg',imgGray)

True

In [ ]:
#数据扩增:对图片做变换，翻转缩放颜色亮度等变化，增加数据，
#常用数据扩增库：
#torchvision 
# https://github.com/pytorch/vision
# pytorch官方提供的数据扩增库，提供了基本的数据数据扩增方法，可以无缝与torch进行集成；但数据扩增方法种类较少，且速度中等；
#imgaug 
#https://github.com/aleju/imgaug
#imgaug是常用的第三方数据扩增库，提供了多样的数据扩增方法，且组合起来非常方便，速度较快；
#albumentations
#https://albumentations.readthedocs.io
#是常用的第三方数据扩增库，提供了多样的数据扩增方法，对图像分类、语义分割、物体检测和关键点检测都支持，速度较快。
'''
transforms.CenterCrop 对图片中心进行裁剪
transforms.ColorJitter 对图像颜色的对比度、饱和度和零度进行变换
transforms.FiveCrop 对图像四个角和中心进行裁剪得到五分图像
transforms.Grayscale 对图像进行灰度变换
transforms.Pad 使用固定值进行像素填充
transforms.RandomAffine 随机仿射变换
transforms.RandomCrop 随机区域裁剪
transforms.RandomHorizontalFlip 随机水平翻转
transforms.RandomRotation 随机旋转
transforms.RandomVerticalFlip 随机垂直翻转
'''

In [10]:
#Pytorch读取数据
#Dataset封装数据集，配置DataLoder【设置 迭代器 】封装批量读取迭代方法


In [11]:
# Pytorch安装,参考https://www.cnblogs.com/zhouzhiyao/p/11784055.html
# 1、创建虚拟环境
# conda create -n pytorch python=3.6
# //安装后查看虚拟环境
# conda info --envs
# 3、激活
# conda activate pytorch
# 4、安装pytorch
# 到官网，自行选择电脑配置生成安装路径，不用装CUDA，选好自动配好代码
# //包windows建议conda，linux建议选pip
# //pytorch Build 选stable
# //cuda 显卡 有GPU的话 python3.6选9.2，没有选None
# //生成结果：conda install pytorch torchvision cudatoolkit=9.2 -c pytorch -c defaults -c numba/label/dev
# 粘到conda下pytorch虚拟环境安装即可


In [59]:
import os,sys,glob,shutil,json
os.environ["CUDA_VISIBLE_DEICES"] = '0'
import cv2
from PIL import Image
import numpy as np
import torch
from torch.utils.data.dataset import Dataset
import torchvision.transforms as transforms
import pickle

In [53]:
#Dataset封装数据集
class SVHNDataset(Dataset):
    def __init__(self,img_path,img_label,transform = None):#__init__后面__别忘了
        self.img_path = img_path
        self.img_label = img_label
        if transform is not None:
            self.transform = transform
        else:
            self.transform = None
            
    def __getitem__(self,index):
        img = Image.open(self.img_path[index]).convert('RGB')#pillow包 按索引打开图片
        if self.transform is not None:
            img = self.tranform(img)#变形？
        #//分10类 SVHN中类别10是0
        #设置最长字符长度为5-观察图一般5个字符，不够pad
        #按索引读数据结果标签
        lbl = np.array(self.img_label[index],dtype = np.int)
        #5-字符长 乘除的数字个数*[10]？
        lbl = list(lbl)+(5-len(lbl))*[10]
        #torch.from_numpy numpy中的ndarray转为torch的tensor，反向操作.numpy()
        return img, torch.from_numpy(np.array(lbl[:5]))
    
    def __len__(self):
        return len(self.img_path)
            
            
            

In [45]:
#配置DataLoder 【设置 迭代器 】批量读取迭代方法['']
# glob.glob返回所有匹配的文件路径列表
train_path = glob.glob('data/mchar_train/*.png')
#print(train_path)#['data/mchar_train\\000000.png', 'data/mchar_train\\000001.png',
train_path.sort()
train_json = json.load(open('data/mchar_train.json'))
#print(train_json['000000.png'])#{'height': [219, 219], 'label': [1, 9], 'left': [246, 323], 'top': [77, 81], 'width': [81, 96]}
train_label = [train_json[x]['label'] for x in train_json]
print(train_label[:2])#[[1, 9], [2, 3]]
print(len(train_path),len(train_label))#查看x,y长度是否匹配

[[1, 9], [2, 3]]
30000 30000


In [52]:
train_loader = torch.utils.data.DataLoader(
    SVHNDataset(
        train_path,train_label,
        transforms.Compose([        #变形组件
            transforms.Resize((64,128)),
            transforms.RandomCrop((60,120)),#随机区域剪裁
            transforms.ColorJitter(0.3,0.3,0.2),#颜色的对比度、饱和度、亮度变换
            transforms.RandomRotation(10),#随机旋转
            transforms.ToTensor(),#ndarray变tensor
            transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])#归一化
        ])),
    batch_size = 40,#一组（批）多少个图（样本）
    shuffle = True,#打乱
    num_workers = 10,#读取的线程个数
    
)
    
# for data in train_loader:
#     break

In [ ]:
#调用DataLoder后，数据按照批次获取，每批次调用Dataset读取单个样本进行拼接
#data格式为x [10,3,64,128] 即[batchsize*chanel*height*width]y [10,6] [batchsize*标签结果值]

In [42]:
#验证集，同上处理
val_path = glob.glob('data/mchar_val/*.png')
val_path.sort()
val_json = json.load(open('data/mchar_val.json'))
val_label = [val_json[x]['label'] for x in val_json]
print(len(val_path),len(val_label))#查看x,y长度是否匹配

10000 10000


In [48]:
val_loader = torch.utils.data.DataLoader(
    SVHNDataset(
        val_path,val_label,
        transforms.Compose([        #变形组件
            transforms.Resize((60,120)),
            #transforms.RandomCrop((60,120)),#随机区域剪裁
            #transforms.ColorJitter(0.3,0.3,0.2),#颜色的对比度、饱和度、亮度变换
            #transforms.RandomRotation(10),#随机旋转
            transforms.ToTensor(),#ndarray变tensor
            transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])#归一化
        ])),
    batch_size = 40,#一组（批）多少个图（样本）
    shuffle = False,#打乱
    num_workers = 10,#读取的线程个数
)

In [58]:
print(len(train_loader))
print(len(val_loader))
print(train_loader)
print(val_loader)

750
250
